# Julia and structural geology 
Started 2019-07-05

This is a preliminar notebook starting investigating the tools available in Julia for structural geology and field mapping processings, in order to acquire experience with this relatively new language.

To start with, we consider the sources listing packages of interest for structural geology and field mapping.
Available sites are:
- Julia Observer: https://juliaobserver.com/packages
- JuliaGeo: https://github.com/JuliaGeo

I first consider the GDAL package, available at https://github.com/JuliaGeo/GDAL.jl

It can be added with the following commands:

In [1]:
using Pkg
Pkg.add("GDAL")
Pkg.test("GDAL")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /home/mauro/.julia/registries/General — failed to fetch from repo
└ @ Pkg.API /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:157


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
   Testing GDAL
    Status `/tmp/tmpg8qAOr/Manifest.toml`
  [b99e7846] BinaryProvider v0.5.4
  [add2ef01] GDAL v0.2.0
  [2a0f44e3] Base64  [`@stdlib/Base64`]
  [8ba89e20] Distributed  [`@stdlib/Distributed`]
  [b77e0a4c] InteractiveUtils  [`@stdlib/InteractiveUtils`]
  [8f399da3] Libdl  [`@stdlib/Libdl`]
  [37e2e46d] LinearAlgebra  [`@stdlib/LinearAlgebra`]
  [56ddb016] Logging  [`@stdlib/Logging`]
  [d6f4376e] Markdown  [`@stdlib/Markdown`]
  [9a3f8284] Random  [`@stdlib/Random`]
  [ea8e919c] SHA  [`@stdlib/SHA`]
  [9e88b42a] Serialization  [`@stdlib/Serialization`]
  [6462fe0b] Sockets  [`@stdlib/Sockets`]
  [8dfed614] Test  [`@stdlib/Test`]


┌ Info: GDAL 2.2.4, released 2018/03/19
│ 159 GDAL drivers found
└ 46 OGR drivers found


   Testing GDAL tests passed 


CPLDestroyMutex: Error = 16 (Device or resource busy)


In [2]:
using GDAL